# AI CA2
# Amirhossein Alizad 810197546
# Genetic Algorithms


<strong>in this computer assignment we are going to use Genetic algorithms and improving them</strong><br>
<strong>we are going to use crossover and mutation algorithms on our generation and create a new generation which should be better than our last generation of chromosomes</strong><br>
<strong>for this assignment we should find a set of gates that the output of their combination should match every output in our file</strong><br>
<strong>first we need to define some concepts</strong><br>


### chromosome:
<strong>a set of gates, we have 6 kinds of gates, AND-OR-XOR-NAND-NOR-XNOR and in this case, each chromosome consists of 9 gates</strong><br>
<br>
### gene:
<strong> each gate in a chromosome is called a gene. genes are given a number between 1 to 6.<br>
1 : AND<br>
2 : OR<br>
3 : XOR<br>
4 : NAND<br>
5 : NOR<br>
6 : XNOR<br>
</strong>
<br>
### first population:
<strong> a set of chromosome which we told in this problem each chromosome has 9 gates. the size of our population is 2<sup>gates-1</sup> and in this problem, 256.</strong><br>
<br>
### fitness: 
<strong> here we considerd fitness the number of outputs which are the same as  given in our file.<br>
for example for a chromosome we test all th set of inputs and when the output is the same as output given in file we increase the fitness<br>
here we powered the fitness by 5 for higher possibilities.</strong>
<br>

### probability:
<strong> shows the possibility of each chromosome. we showed this with 'p'.<br>
$P_i$ = $fitness_i$ / $\sum_j^n fitness_j$<br></strong>
<br>


In [1]:
import pandas as pd
import random
import time
import copy
from numpy.random import choice
from numpy import arange

In [2]:
def AND(in1, in2):
    return (in1 & in2)

def OR(in1, in2):
    return (in1 | in2)

def XOR(in1, in2):
    return (in1 ^ in2)

def NAND(in1, in2):
    return ~(in1 & in2)

def NOR(in1, in2):
    return ~(in1 |in2)

def XNOR(in1, in2):
    return ~(in1 ^ in2)

def fitness_calc(population, df):
    df_pop = pd.DataFrame(population)
    df_fit = df_pop.apply(lambda x: calc_fitness(df, x), axis = 1)
    fitness = df_fit.tolist()
    return [x**5 for x in fitness]

def calc_fitness(df, row):
    lst = [j for j in row]
    tmp = [df[i] for i in df]
    ans = [False for i in range(len(lst) + 1)]
    ans[0] = tmp.pop(0)
    out = tmp.pop()
    for i in range(len(lst)):
        if lst[i] == 1:
            ans[i + 1] = AND(tmp[i], ans[i])
        elif lst[i] == 2:
            ans[i + 1] = OR(tmp[i], ans[i])  
        elif lst[i] == 3:
            ans[i + 1] = XOR(tmp[i], ans[i])
        elif lst[i] == 4:
            ans[i + 1] = NAND(tmp[i], ans[i])
        elif lst[i] == 5:
            ans[i + 1] = NOR(tmp[i], ans[i])
        else:
            ans[i + 1] = XNOR(tmp[i], ans[i])
    return (ans[-1] == out).sum()


def crossover(ch1,ch2,gates):
    sep = random.randint(1,gates - 1)
    ans1 = ch1[ : sep + 1] + ch2[sep + 1 : ]
    ans2 = ch2[ : sep + 1] + ch1[sep + 1 : ]
    return ans1, ans2

def create_gen(population, gates, probability, pc, pm):
    new_pop = []
    length = len(population)/2
    i = 0
    while(i < length):
        idx = choice(arange(0, len(population)),2,replace=False, p=probability)
        new_gen1 = population[idx[0]]
        new_gen2 = population[idx[1]]
        if random.random() < pc:
            new_gen1, new_gen2 = crossover(population[idx[0]], population[idx[1]], gates)
            if random.random() < pm:
                new_gen1[random.randint(0,gates - 1)] = random.randint(1,6)
            if random.random() < pm:
                new_gen2[random.randint(0,gates - 1)] = random.randint(1,6)
        new_pop.append(new_gen1)
        new_pop.append(new_gen2)
        i += 1
    return new_pop


<strong> as shown above first we chose 2 random numbers according to our possibility(as we discussed before).<br>
then we generate a random number between 0 an 1 and if it was less than $P_c$ we start crossover and if it wasn't, then we move those two chromosomes to our next generation.</strong><br>
<strong> for crossover we first choose a random index then we start  crossing the chromosomes over. we used 1-point crossover here.<br>
after we performed crossover on our chromosomes, then we perform mutation on them with the possibility of $P_m$.<br>
to perform mutation we generate a random number between 0 and 1 and if it was less than $P_m$ then we perform mutation and if it wasn't, then we don't. we do this for both of the chosen chromosomes after performing crossover on them.<br></strong>
<br>

In [3]:
df = pd.read_csv('truth_table.csv')
gates = len(df.columns) - 2
gate_dic = {1 : "AND", 2: "OR", 3 : "XOR", 4 : "NAND", 5 : "NOR", 6 : "XNOR"}
count = 2 ** (gates-1)
population = [[] for i in range(count)]
length = len(population)
for i in range(count):
    for j in range(gates):
        population[i].append(random.randint(1, 6))
fitness = fitness_calc(population, df)
s = sum(fitness)
p = [0 for i in range(len(fitness))]
for i in range(len(fitness)):
    p[i] = fitness[i] / s

In [4]:
pc = 0.8
pm = 0.1
last_gen = population
last_max = -1
t1 = time.time()
while True:
    new_gen = create_gen(last_gen, gates, p, pc, pm)    
    fitness = fitness_calc(new_gen, df)
    s = sum(fitness)
    for i in range(len(fitness)):
        p[i] = fitness[i] / s
    if max(fitness) == last_max:
        pc += 0.01
        pm += 0.01
    if(max(fitness) == (2**(gates + 1)) ** 5):
        idx = fitness.index(max(fitness))
        print("Gates : ", end = "")
        for i in new_gen[idx]:
            print(gate_dic[i], end = " ")
        break
    last_max = max(fitness)
    last_gen = copy.deepcopy(new_gen)
print()
print(time.time() - t1, "seconds")


Gates : NAND XNOR OR XNOR AND OR NAND XNOR NOR 
4.349992513656616 seconds


<strong>we repeat the steps discussed above untill we find a certain answer that matches all the outputs in the file.<br>
after testing diffrent possibilities we concluded that the best choice for $P_c$ is between 0.75 and 0.85.
and it is 0.1 for $P_m$ as it should be much less than $P_c$.<br>
every time we have a generation like the last one, we simply increase our possibilities by 0.01.</strong>

# پرسش ها 
۱)
دلیل انتخاب معیار تناسب ما این است که ما باید مجموعه ای را پیدا کنیم که تمام خروجی های آن به ازای ورودی های داده شده با خروجی های موجود در فایل مورد نظر یکسان باشد.و دلیل اینکه ما توانی از ۵ این مقدار را انتخاب کرده ایم این است که احتمال انتخاب هر کروموزومی که به جواب نزدیکتر است بیشتر میشود و اینکه تغییرات نسل قبلی با نسل بعدی به صورت خطی نخواهد بود

۲)
با احتمال داده شده به تابع ۲ نفر از افراد جمعیت را انتخاب کرده و روی آنها الگوریتم های ذکر شده را پیاده سازی میکنیم.


۳)
crossover:
این عمل باعث میشود که اگر قسمتی از ژن های انتخاب شده مناسب باشند و قسمتی دیگر نامناسب احتمال اینکه قسمت نامناسب دور انداخته شود بالااست و با این کار قسمتی که از نظر ما مناسب است با قسمتی از کروموزوم دیگر ترکیب میشود که مارا به جواب نزدیکتر میکند
mutation:
این عمل پس از هر بار کراس اوور انجام میشود به این دلیل که ممکن است هر بار پس از انجام این عملیات در حالت خاصی گیر کرده باشیم با تغییر  جزئی روی این کروموزوم ها میتوان از این حالت بیرون آمد و به جواب نزدیکتر شد


۴)
در صورتی که این اتفاق برای ما پیش بیاید به این معنی است که در یک نقطه ی ماکسیمم محلی گیر افتاده ایم و باید از این نقطه بیرون بیاییم.برای این کار ما کافی است که احتمال های 
<br>pc, pm <br>
را مقداری بیشتر کنیم که با تست های انجام شده مقدار مناسب برای این کار 0.01 است